In [1]:
import os
import click
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from trainers import RewardModelTrainer
from configs import get_configs
from gpt import GPTActor
from dataset import DahoasRMStaticDataset, DahoasSFTStaticPromptsDataset

d:\ProgramData\Anaconda3\envs\RL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ---device = 'cuda'
batch_size = 2
pretrain = './runs/sft_sft_puns_100k_202404290030/params_final.pt'
exp_name = 'rm_test_puns'
cfg = get_configs("gpt2-medium/lora")
cfg.batch_size = batch_size
cfg.pretrain = pretrain
cfg.total_epochs = 1
cfg.exp_name = exp_name

In [3]:
rm = GPTActor.from_checkpoint(cfg, pretrain)

In [4]:
dataset = DahoasSFTStaticPromptsDataset(block_size=1024,
                                            max_examples=None,
                                            tokenizer_name="tiktoken/gpt2")

Generating test split: 100%|██████████| 5103/5103 [00:00<00:00, 318913.09 examples/s]


Loading DahoasSFTStaticPromptsDataset


In [5]:
from torch.utils.data import DataLoader


In [6]:
train_dataloader = DataLoader(dataset,
                                           batch_size=cfg.batch_size,
                                           num_workers=12,
                                           prefetch_factor=4,
                                           pin_memory=True)



In [19]:
max_new_tokens = 100

In [12]:
prompt, input_mask, input_length = next(iter(train_dataloader))

In [16]:
prompt.size()

torch.Size([2, 1024])

In [17]:
input_mask.size()

torch.Size([2, 1024])

In [20]:
completion, attention_mask, num_actions, action_mask = rm.batch_generate(
            prompt, input_mask, input_length, max_new_tokens, temperature=1.0, top_k=50)

In [23]:
from tokenizer import TiktokenTokenizer

tokenizer = TiktokenTokenizer('gpt2')

In [31]:
s = tokenizer.enc.decode(completion[1].tolist())

In [32]:
s

"\n\nHuman: What are some foods that are good for diabetics?\n\nAssistant: To be honest, some of these are better than others, and they’re a little more like opinions than facts. For example, many of the diets say to limit vegetables with high sugar content, and there’s some debate on the subject, as far as how much of these vegetables are actually bad for diabetics.\n\nHuman: Okay, any other advice?\n\nAssistant: Always lend a little to my favorite hotel<|endoftext|>Human: make a pun about lending money\n\nAssistant: Sure thing!\nA: the government isn't doing anything to help us even keep the clothes on our back\nB: the government begged the banks to lint money<|endoftext|>Human: make a pun about window panes\n\nAssistant: Sure thing!\nA: those windows in the dorms are such a pain\nB: gmc will complete the process of replacing single pain windows in its residence halls<|endoftext|>Human: make a pun about mood\n\nAssistant: Sure thing!\nA:<|endoftext|><|endoftext|><|endoftext|><|endoft

In [33]:
s = s[:s.find('<|endoftext|>')]

In [35]:
print(s)



Human: What are some foods that are good for diabetics?

Assistant: To be honest, some of these are better than others, and they’re a little more like opinions than facts. For example, many of the diets say to limit vegetables with high sugar content, and there’s some debate on the subject, as far as how much of these vegetables are actually bad for diabetics.

Human: Okay, any other advice?

Assistant: Always lend a little to my favorite hotel


In [21]:
completion

tensor([[  198,   198, 20490,  ..., 50256, 50256, 50256],
        [  198,   198, 20490,  ..., 50256, 50256, 50256]])

In [11]:
for step, (prompt, input_masks, input_lengths) in enumerate(train_dataloader):
    print('promtp: ', prompt)
    completion, attention_mask, num_actions, action_mask = rm.batch_generate(
            prompt, input_masks, input_lengths, max_new_tokens, temperature=1.0, top_k=50)
    
    break

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001E7065D6950>
Traceback (most recent call last):
  File "d:\ProgramData\Anaconda3\envs\RL\lib\site-packages\torch\utils\data\dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "d:\ProgramData\Anaconda3\envs\RL\lib\site-packages\torch\utils\data\dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


KeyboardInterrupt: 